In [ ]:
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense, Dropout
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.preprocessing import StandardScaler
import seaborn as sns
import tensorflow as tf

In [ ]:
data_f = pd.read_csv("C:\RUTU\WF_project\WF-CMT-6-MBS\Combined_Dataset\Dataset.csv")
print(data_f.head())

In [ ]:
cols=data_f.columns
print(len(cols))
print(cols)

In [44]:
data_f.drop(['EMI','Interest_Component','PREPAYMENT_AMT','Previous_UPB','Remaining MFM'], axis = 1, inplace = True)

In [45]:
cols=data_f.columns
print(len(cols))
print(cols)

32
Index(['Loan Identifier', 'Monthly Reporting Period', 'Channel',
       'Original Interest Rate', 'Current Interest Rate', 'Original UPB',
       'Current Actual UPB', 'Original Loan Term', 'Origination Date',
       'First Payment Date', 'Remaining Months To Maturity',
       'Original Loan to Value Ratio (LTV)', 'Debt-To-Income (DTI)',
       'Borrower Credit Score at Origination',
       'First Time Home Buyer Indicator', 'Loan Purpose', 'Property Type',
       'Number of Units', 'Occupancy Status',
       'Metropolitan Statistical Area (MSA)', 'Zip Code Short',
       'Interest Only Loan Indicator', 'Current Loan Delinquency Status',
       'Modification Flag', 'Zero Balance Code', 'Special Eligibility Program',
       'Property Valuation Method', 'High Balance Loan Indicator',
       'Unemployment_rate', 'Divorce_rate', 'Inflation_rate', 'UPB_diff'],
      dtype='object')


In [77]:
from numpy import array

def split_sequences(sequences, n_steps): 
 
    print(type(sequences))   
    X,y = list(), list()
    for i in range(len(sequences)):
    # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the dataset
        if end_ix > len(sequences):
            break
    # gather input and output parts of the pattern
    seq_x, seq_y = sequences['Loan Identifier'], sequences['Current Actual UPB']
    print(seq_y)
    print(seq_x)    
    
    X.append(seq_x)    
    y.append(seq_y)
    return array(X), array(y)

In [58]:
print(data_f['Current Actual UPB'])

0           75000.00
1           75000.00
2           75000.00
3           75000.00
4           75000.00
             ...    
1256463    500744.85
1256464    499853.61
1256465    498960.10
1256466    498064.31
1256467         0.00
Name: Current Actual UPB, Length: 1256468, dtype: float64


In [78]:
X,Y= split_sequences(data_f, 3)

<class 'pandas.core.frame.DataFrame'>
0           75000.00
1           75000.00
2           75000.00
3           75000.00
4           75000.00
             ...    
1256463    500744.85
1256464    499853.61
1256465    498960.10
1256466    498064.31
1256467         0.00
Name: Current Actual UPB, Length: 1256468, dtype: float64
0          100007365142
1          100007365142
2          100007365142
3          100007365142
4          100007365142
               ...     
1256463       105675700
1256464       105675700
1256465       105675700
1256466       105675700
1256467       105675700
Name: Loan Identifier, Length: 1256468, dtype: int64


In [ ]:
time_steps = 3
n_features = X.shape[2]

In [ ]:
hiddensizes = [15,20, 30, 40]
dropouts = [0,0.8,0.15,0.2]
batch_sizes = [32,64,128]
optimizers = ['rmsprop', 'adam']
for dropout in dropouts:
    for batch_size in batch_sizes:
        for optimizer in optimizers:
            for hidden in hiddensizes:
                print('dropout:' + str(dropout) + '\t' + 'batchsize:' + str(batch_size) + '\t' + 'optimzer:' + str(optimizer)  + '\t' + 'Hidden_U:' + str(hidden))

                lstm_model = Sequential()
                #hiddensize1 = X.shape[2]

                lstm_model.add(LSTM(units = hidden, dropout=dropout, return_sequences = True, input_shape = (time_steps, n_features)))
                lstm_model.add(LSTM(units = hidden, dropout=dropout, return_sequences = True))
                #lstm_model.add(LSTM(units = hiddensize1, dropout=0.2, return_sequences = True))

                lstm_model.add(Dense(units = 4, activation='softmax'))
                weights = np.array([0.005,0.24,0.75,0.005])
                #weights = np.ones((4,))*1.0
                lstm_model.compile(optimizer = optimizer, loss = 'mse', metrics = ['categorical_accuracy'])
                lstm_model.fit(X, Y*1.0,batch_size=batch_size, epochs=4, verbose=1, validation_data=(Xtest, Ytest*1.0))